In [137]:
import requests
import urllib3
import csv
import pandas as pd
import re



from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter

In [138]:
from urllib3.util.ssl_ import create_urllib3_context
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# Custom SSL context to allow smaller DH keys
class SSLAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        context.set_ciphers('DEFAULT@SECLEVEL=1')  # Lowering the security level
        self.poolmanager = urllib3.PoolManager(*args, ssl_context=context, **kwargs)

In [139]:
# Fectch the website u want
def fetch_web_page(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    headers = {'User-Agent': user_agent}

    # Create a session and mount the custom SSL adapter
    session = requests.Session()
    session.mount('https://', SSLAdapter())

    # Make the request with headers
    response = session.get(url, headers=headers, verify='/Users/boruchen/.local/share/virtualenvs/langchain-practice-N2Mm1uHF/lib/python3.11/site-packages/certifi/cacert.pem')
    response.encoding = 'utf-8'
    print(response)

    # Parse the response content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [140]:
# Fetch all the courses' url of a department
def fetch_courses_url(url):
    # Fetching all the courses' name & url
    iim_soup = fetch_web_page(url)

    # Find all td elements containing course information
    course_blocks = iim_soup.find_all('td')
    
    courses_url = []
    course_list = []
    for block in course_blocks:
        if block.find('span', class_='course_name'):
            course_name = block.find('span', class_='course_name').getText().strip()
            course_url = ""
            if block.find('a'):  # Directly look for 'a' tag under 'block'
                a_tag = block.find('a')
                course_url = a_tag['href'].strip()
            # Check if the course name is already added to avoid duplicates
            if course_name and course_name not in course_list:
                course_list.append(course_name)
                if course_url: 
                    courses_url.append(course_url)
                print("Course Name:", course_name)
                print("Course Url:", course_url)
                print()

    print(course_list)
    return courses_url
    

In [141]:
def fectch_syllabus_url(url):
    course_soup = fetch_web_page(url)
    course_elements = course_soup.find_all('td')
    newest_syllabus_url = course_elements[2].find('a')['href'].strip()

    return newest_syllabus_url

In [142]:
test_url = fectch_syllabus_url('https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H2&cono=H211620')

<Response [200]>


In [143]:
print(fetch_syllabus_page(test_url))

<Response [200]>
{'Course Name': '統計學（二）', 'Grading': '方法\t百分比%\n作業 Assignments\t10\n期中考 Midterm Exam\t20\n期中考 Midterm Exam\t30\n期末考 Term exam\t40', 'Department': '統計系Statistics', 'Teaching Strategies': '方法\t百分比%\n講授 Lecture\t100', 'Syllabus Progress': '進度說明 Progress Description\n1\tReview Chapters 1-8\n2\tReview Chapters 1-8 & Tests of Hypotheses-1\n3\tTests of Hypotheses- 2 and Review test\n4\tInference two population means and proportions -1\n5\tInference two population means and proportions -2\n6\tReviews and Test 1\n7\tInference one and two population variances\n8\tCategorical Data Analysis\n9\tDesign of Experiments & Analysis of Variance.-1\n10\tDesign of Experiments & Analysis of Variance.-2\n11\tReviews\n12\tReviews and test 2\n13\tSimple Linear Regression -1\n14\tSimple Linear Regression -2\n15\tMultiple Regression -1\n16\tMultiple Regression -2\n17\tReviews\n18\tFinal Exam', 'Course Description': '課程概述 Course Description\n       統計學之意義與應用、統計資料之搜集及圖表、敘述統計、基本機率概念、期望值、機率分配、抽樣分配、

In [144]:
# Fetch the syllabus url and content
def fetch_syllabus_page(url):
    
    syllabus_soup = fetch_web_page(url)
    syllabus_content = syllabus_soup.find('div', id = 'content')
    header_div = syllabus_soup.find('div', id='header')

    # 找到header_div中的所有span元素
    span_elements = header_div.find_all('span')

    # 选择列表中的第二个span元素
    second_span_element = span_elements[1]

    # 找到second_span_element中的br元素的下一个兄弟节点，即br后面的文本内容
    course_name = second_span_element.find('br').next_sibling.strip()

    syllabus_div_elements = syllabus_content.find_all('div', class_ = 'block')
    syllabus_description = syllabus_div_elements[2].getText().strip()
    syllabus_progress = syllabus_div_elements[4]



    side_bar_elements = syllabus_soup.find('div', id = 'sidebar')

    # 使用已經存在的 side_bar_elements 變數
    basic_data_div = side_bar_elements.find('div', class_='block basic_data')


    # 從基本資料區塊中抓取開課系所資訊
    department = basic_data_div.find('span', string='開課系所 Department/Institute:').next_sibling.strip()



    grading_div = side_bar_elements.find_all('div', class_='block')[6]
    teaching_strats_div = side_bar_elements.find_all('div', class_='block')[7]


    def fetch_table_data(div):
        table = div.find('table')
        if table:    
            tbody = table.find('tbody')
            headers = []
            for th in tbody.find_all('th'):
                headers.append(th.text.strip())
            rows = []
            for tr in tbody.find_all('tr'):
                row = []
                # 处理th元素
                for th in tr.find_all('th'):
                    row.append(th.text.strip())
                # 处理td元素
                if len(tr.find_all('td')) <= 2:
                    for td in tr.find_all('td'):
                        row.append(td.text.strip())
                    rows.append(row)
            return headers, rows
        else:
            print("error")
            return None, None

    # 调用函数获取数据
    grading_headers, grading_rows = fetch_table_data(grading_div)
    teaching_strats_headers, teaching_strats_rows = fetch_table_data(teaching_strats_div)
    syllabus_progress_headers, syllabus_progress_rows = fetch_table_data(syllabus_progress)

    # 合并表格数据为字符串
    def table_to_string(headers, rows, include_headers=True):
        if not headers or not rows:
            return ""
        table_str = ""
        for row in rows:
            table_str += "\t".join(row) + "\n"
        return table_str.strip()

    grading_str = table_to_string(grading_headers, grading_rows, include_headers=True)
    teaching_strats_str = table_to_string(teaching_strats_headers, teaching_strats_rows, include_headers=True)
    syllabus_progress_str = table_to_string(syllabus_progress_headers, syllabus_progress_rows, include_headers=True)




    course_info = {
        'Course Name': course_name,
        'Grading': grading_str,
        'Department': department,
        'Teaching Strategies': teaching_strats_str,
        'Syllabus Progress': syllabus_progress_str,
        'Course Description': syllabus_description
    }

    return course_info



In [145]:
# 打開文件讀取個系所的課程列表 url
with open('department_urls.txt', 'r') as file:
    # 讀取文件內容
    content = file.read()


# 使用正則表達式找到所有的課程列表網址
department_urls = re.findall(r'https?://\S+', content)
print(department_urls)

['https://course-query.acad.ncku.edu.tw/index.php?c=qry11215&i=UjIDagU1B2oIIAVyVj1ValY7A3RVOlQrAjhcOlRpCTcBMFZuXTwIclA9AGIEOlZzAGIPfwZpVTcGMgthAT1QLgdvVTwANAVpVGYAPlMwAjYBLVJ0V2sAaAI5DSAFcV4oCWwJaFx0ASpTP1BqXmBXYVdnAmJUO1cnAjYOY1ZqDCtSIAM1BWwHcghgBXJWPVVqVjsDdFViVCsCOFwuVDoJPAEyVm1dZgg7UDYAaQR7ViIAOw9qBmhVdAZsCzABd1AhB1lVaABrBXBUPAB3UzgCOQFsUiVXbwB6AkINOwUlXnMJMgl0XG8BMlM1UDJealdrVzcCalQyV24Cdw4jVmoMOlJpA3IFcgcpCCgFI1Y8VShWOwNjVWlUKwJ2XD1UNwljAXNWdF08CHJQPQBkBDpWcwBiDzIGPlU6Bm0LMgFiUHcHPVV1AD4FY1Q9ACZTUAIuAW1Selc4AGcCMg0zBTdeagloCTNcNwEyU39QcF5gV2dXbAJyVGRXMQJ8DiRWDwxnUjwDcgU6ByMIYQUzVj1VeVZJA2dVcVQyAn5cLlQgCTwBMFZsXSUIc1AlAGgEYVZrADQPZwYpVT8GMgtlATxQPAc8VTYAPwU7VD0ANVM5AmUBbFI0V2oANAI5DTAFOV44CTMJOFw2AThTP1BqXmBXY1dtAjlUOldlAj0OYFZrDGBSaQNkBToHOQhhBTBWNlVuVjoDK1UgVDMCMlxpVHEJYQEjVm1dZQg7UDcAaAR9', 'https://course.ncku.edu.tw/index.php?c=qry11215&i=VWReZQBgBTpVKgF4BGdSOwQ4Wi8EbwMhVmoAPAo2U2lRYlFvUzhWIwNpUDBVPAAtVmIEIAk3AmJTaVQ1UmtdLQ82WWIGYlFvUTQCMgJqB2VRLFZyUWkBNgA9XSoEZlY4ViRSIAoFVj4

In [146]:
def save_courses_data_to_json(courses_data_list):
    """
    將多個課程資料保存為單一 JSON 文件的函數。

    Parameters:
        courses_data_list (list of lists): 包含多個課程資料的列表，每個元素都是一個課程資料列表。

    Returns:
        None
    """
    # 合併多個 courses_data 到一個列表中
    combined_data = []
    for data in courses_data_list:
        combined_data.extend(data)
    
    # 將課程資料轉換為 DataFrame
    df = pd.DataFrame(combined_data)
    
    # 將 DataFrame 轉換為 JSON 格式
    json_data = df.to_json(orient='records', force_ascii=False, indent=4)
    
    # 將 JSON 寫入文件
    with open('courses_data.json', 'w', encoding='utf-8') as f:  # 使用 'w' 模式重新寫入文件，而不是 'a'
        f.write(json_data)
    
    print("JSON 文件已创建")


In [147]:
def get_courses_data(courses_url):
    # 抓取所有课程数据
    courses_data = []
    for url in courses_url:
        course_info = fetch_syllabus_page(fectch_syllabus_url(url))
        if course_info:
            courses_data.append(course_info)
    return courses_data

In [148]:

# 創建一個空列表來存儲所有的 courses_data
all_courses_data = []

# 遍歷每個 department_url
for url in department_urls:
    courses_url = fetch_courses_url(url)
    courses_data = get_courses_data(courses_url)
    # 將每個 courses_data 添加到 all_courses_data 列表中
    all_courses_data.append(courses_data)

# 將包含所有 courses_data 的列表傳遞給 save_courses_data_to_json 函數
save_courses_data_to_json(all_courses_data)


<Response [200]>
Course Name: 外國語言（一）
Course Url: 

Course Name: 初級會計學（一）
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H111710

Course Name: 大學國文（一）
Course Url: 

Course Name: 微積分（一）
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H115611

Course Name: 經濟學（一）
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H111210

Course Name: 民法概要
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H111100

Course Name: 體育（一）
Course Url: 

Course Name: 初級會計學
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H112200

Course Name: 通識課程
Course Url: 

Course Name: 服務學習（一）
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H117910

Course Name: 統計學（一）
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=H1&cono=H120310

Course Name: 中級會計學（一）
Course Url: https://c